# Delta Practice

### File Read In

In [18]:
import pandas as pd
from datetime import timedelta

filepath1 = r'/Users/cartersocha/Downloads/instgramHashtagCounts.xlsx'
instaHashtagDf = pd.read_excel(filepath1)

filepath2 = r'/Users/cartersocha/Downloads/tweetCountTest.xlsx'
dailyTweetDf = pd.read_excel(filepath2)

filepath3 = r'/Users/cartersocha/Desktop/ReleaseData.xlsx'
releaseDf = pd.read_excel(filepath3, "ShowInfoEndStart")

filepath4 = r'/Users/cartersocha/Downloads/instgramAccountCounts.xlsx'
igAccountDf = pd.read_excel(filepath4)

filepath5 = r'/Users/cartersocha/Downloads/redditCountTest.xlsx'
redditSubsDf = pd.read_excel(filepath5)

filepath6 = r'/Users/cartersocha/Downloads/redditCommentCombo.csv'
redditCommentsDf = pd.read_csv(filepath6)

filepath8 = r'/Users/cartersocha/Desktop/ReleaseData.xlsx'
releaseDateDf = pd.read_excel(filepath8, "ReleaseDateData")

filepath10 = r'/Users/cartersocha/Downloads/googleDataset2.csv'
googleDailyData = pd.read_csv(filepath10)

### Functions

In [19]:
def DateTimeConvert(dateDf, dateColumn):
    dateDf[dateColumn] = pd.to_datetime(dateDf[dateColumn])  

    return dateDf

In [20]:
def DiffMaker(fillnaDf, valueColumn, dateColumn):
    fillnaDf.sort_values(['TvShow', dateColumn], inplace=True)

    fillnaDf['diffs'] = fillnaDf.groupby(['TvShow'])[valueColumn].transform(lambda x: x.diff()).fillna(0)

    fillnaDf.sort_index(inplace=True)

    return fillnaDf

In [21]:
def RemoveData(release,showDf):

    bigDf = pd.DataFrame()

    for i in range(len(release)):
        show = release['TvShow'][i]
        firstDate = release['Release Date'][i]
        secondDate = release['90DayDate'][i]

        smallDf = showDf[showDf['TvShow'] == show]

        newdf = smallDf[smallDf['RunDate'].between(firstDate, secondDate)]

        bigDf = bigDf.append(newdf,ignore_index=True)

    return bigDf

In [22]:
def MergeDfs(mainDf, secondDf, columnName):

    merged = pd.merge(mainDf,secondDf, how='outer', on=columnName)
    return merged

In [23]:
import numpy as np

def NegativeDiffs(diffDf,columnName):

    diffDf['zeroedDiffs'] = np.where((diffDf[columnName] < 0), 0, diffDf[columnName])

    return diffDf

In [24]:
def SumSocialMedia(sumDf,columnName,trigger):
    # check to see if this is episode count or generic summarization
    if trigger == 1:
        summarizedDf = sumDf.groupby(columnName, as_index=False).sum()
        summarizedDf = pd.DataFrame(summarizedDf)
    
    else:
        summarizedDf = sumDf.groupby(columnName, as_index=False).count()
        summarizedDf = pd.DataFrame(summarizedDf)
    
    return summarizedDf

In [25]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PowerTransformer

def NormalizeData(normalDf,columnName):

    # define min max scaler
    scaler = MinMaxScaler()
    scaler1 = StandardScaler()
    scaler2 = MaxAbsScaler()
    scaler3 = RobustScaler()
    scaler4 = Normalizer()
    scaler5 = QuantileTransformer()
    scaler6 = PowerTransformer()

    # transform data
    normalDf['ScaledDataAdjust-MinMax'] = scaler.fit_transform(normalDf[[columnName]])
    normalDf['ScaledDataAdjust-StandardScaler'] = scaler1.fit_transform(normalDf[[columnName]])
    normalDf['ScaledDataAdjust-MaxAbsScaler'] = scaler2.fit_transform(normalDf[[columnName]])   
    normalDf['ScaledDataAdjust-RobustScaler'] = scaler3.fit_transform(normalDf[[columnName]])   
    normalDf['ScaledDataAdjust-Normalizer'] = scaler4.fit_transform(normalDf[[columnName]])   
    normalDf['ScaledDataAdjust-QuantileTransformer'] = scaler5.fit_transform(normalDf[[columnName]])
    normalDf['ScaledDataAdjust-PowerTransformer'] = scaler6.fit_transform(normalDf[[columnName]])       
    
    return normalDf

### Data Transformations

##### Release Data

In [26]:
releaseDf['90DayDate'] = releaseDf['Release Date'] + pd.DateOffset(days=120)

releaseDf = DateTimeConvert(releaseDf,'90DayDate')
releaseDf = DateTimeConvert(releaseDf,'Release Date')

##### Instagram Hashtag Data

In [27]:
instaHashtagDf = DateTimeConvert(instaHashtagDf,'RunDate')

instaHashtagDf = DiffMaker(instaHashtagDf,'HashtagValue','RunDate')

instaHashDf = RemoveData(releaseDf, instaHashtagDf)
instaHashDf = NegativeDiffs(instaHashDf, 'diffs')
instaHashDf['SocialMediaSource'] = 'InstagramHashtag'
instaHashDf['SocialMediaValue'] = instaHashDf['zeroedDiffs']

instaHashDf = NormalizeData(instaHashDf,'SocialMediaValue')

/var/folders/7v/2b53j97121jg00pcthbt6kz80000gn/T/ipykernel_53460/2781502079.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  bigDf = bigDf.append(newdf,ignore_index=True)


##### Twitter Data

In [28]:
dailyTweetDf = DateTimeConvert(dailyTweetDf,'RunDate')
dailyTweetDf = RemoveData(releaseDf, dailyTweetDf)
dailyTweetDf['SocialMediaValue'] = dailyTweetDf['TweetCount'] + dailyTweetDf['RetweetCount']
dailyTweetDf['SocialMediaSource'] = 'Tweets'
dailyTweetDf = NormalizeData(dailyTweetDf,'SocialMediaValue')

/var/folders/7v/2b53j97121jg00pcthbt6kz80000gn/T/ipykernel_53460/2781502079.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  bigDf = bigDf.append(newdf,ignore_index=True)


##### Reddit Comment Data

In [29]:
redditCommentsDf = DateTimeConvert(redditCommentsDf,'RunDate')
redCommentDf = RemoveData(releaseDf, redditCommentsDf)
redCommentDf['SocialMediaValue'] = redCommentDf['NumComments'] + redCommentDf['score']

redCommentDf = redCommentDf[['TvShow','RunDate','SocialMediaValue']]
iRedCommentDf = SumSocialMedia(redCommentDf, ['TvShow','RunDate'],1)
iRedCommentDf['SocialMediaSource'] = 'RedditComments'

iRedCommentDf = NormalizeData(iRedCommentDf,'SocialMediaValue')

/var/folders/7v/2b53j97121jg00pcthbt6kz80000gn/T/ipykernel_53460/2781502079.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  bigDf = bigDf.append(newdf,ignore_index=True)


##### Instagram Account Data

In [30]:
igAccountDf = DateTimeConvert(igAccountDf,'RunDate')

igAccountDf = DiffMaker(igAccountDf,'IgAccountCounts','RunDate')


instaAccDf = RemoveData(releaseDf, igAccountDf)
instaAccDf = NegativeDiffs(instaAccDf, 'diffs')
instaAccDf['SocialMediaSource'] = 'InstagramAccount'
instaAccDf['SocialMediaValue'] = instaAccDf['zeroedDiffs']

instaAccDf = NormalizeData(instaAccDf,'SocialMediaValue')

/var/folders/7v/2b53j97121jg00pcthbt6kz80000gn/T/ipykernel_53460/2781502079.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  bigDf = bigDf.append(newdf,ignore_index=True)


##### Reddit Subscriber Data

In [31]:
redditSubsDf = DateTimeConvert(redditSubsDf,'RunDate')

redditSubsDf = DiffMaker(redditSubsDf,'RedditSubs','RunDate')

redSubDf = RemoveData(releaseDf, redditSubsDf)
redSubDf = NegativeDiffs(redSubDf, 'diffs')
redSubDf['SocialMediaSource'] = 'RedditSubscribers'
redSubDf['SocialMediaValue'] = redSubDf['zeroedDiffs']

redSubDf = NormalizeData(redSubDf,'SocialMediaValue')

/var/folders/7v/2b53j97121jg00pcthbt6kz80000gn/T/ipykernel_53460/2781502079.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  bigDf = bigDf.append(newdf,ignore_index=True)


##### Google Data

In [32]:
googleDailyData = DateTimeConvert(googleDailyData,'RunDate')
googDf1 = RemoveData(releaseDf, googleDailyData)
googDf1 = NegativeDiffs(googDf1, 'GoogleValue')

googDf1['SocialMediaSource'] = 'Google'
googDf1['SocialMediaValue'] = googDf1['zeroedDiffs']

googDf1 = NormalizeData(googDf1,'SocialMediaValue')

/var/folders/7v/2b53j97121jg00pcthbt6kz80000gn/T/ipykernel_53460/2781502079.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  bigDf = bigDf.append(newdf,ignore_index=True)


##### Release Date Data

In [33]:
releaseDateDf = DateTimeConvert(releaseDateDf,'EpisodeReleaseDate')
iReleaseData = SumSocialMedia(releaseDateDf, ['TvShow','EpisodeReleaseDate'],0)
iReleaseData['DailyReleaseCount'] = iReleaseData['EpisodeNumber']
iReleaseData['RunDate'] = iReleaseData['EpisodeReleaseDate']

##### Join the Data

##### Time Value Datasets

In [34]:
instaAccDf = instaAccDf[['TvShow','RunDate','SocialMediaValue','SocialMediaSource','ScaledDataAdjust-MinMax','ScaledDataAdjust-StandardScaler',
'ScaledDataAdjust-MaxAbsScaler','ScaledDataAdjust-RobustScaler','ScaledDataAdjust-Normalizer','ScaledDataAdjust-QuantileTransformer'
,'ScaledDataAdjust-PowerTransformer']]#,'diffs']]
redSubDf = redSubDf[['TvShow','RunDate','SocialMediaValue','SocialMediaSource','ScaledDataAdjust-MinMax','ScaledDataAdjust-StandardScaler',
'ScaledDataAdjust-MaxAbsScaler','ScaledDataAdjust-RobustScaler','ScaledDataAdjust-Normalizer','ScaledDataAdjust-QuantileTransformer'
,'ScaledDataAdjust-PowerTransformer']]#,'diffs']]
dailyTweetDf = dailyTweetDf[['TvShow','RunDate','SocialMediaValue','SocialMediaSource','ScaledDataAdjust-MinMax','ScaledDataAdjust-StandardScaler',
'ScaledDataAdjust-MaxAbsScaler','ScaledDataAdjust-RobustScaler','ScaledDataAdjust-Normalizer','ScaledDataAdjust-QuantileTransformer'
,'ScaledDataAdjust-PowerTransformer']]#,'diffs']]
instaHashDf = instaHashDf[['TvShow','RunDate','SocialMediaValue','SocialMediaSource','ScaledDataAdjust-MinMax','ScaledDataAdjust-StandardScaler',
'ScaledDataAdjust-MaxAbsScaler','ScaledDataAdjust-RobustScaler','ScaledDataAdjust-Normalizer','ScaledDataAdjust-QuantileTransformer'
,'ScaledDataAdjust-PowerTransformer']]#,'diffs']]
googDf1 = googDf1[['TvShow','RunDate','SocialMediaValue','SocialMediaSource','ScaledDataAdjust-MinMax','ScaledDataAdjust-StandardScaler',
'ScaledDataAdjust-MaxAbsScaler','ScaledDataAdjust-RobustScaler','ScaledDataAdjust-Normalizer','ScaledDataAdjust-QuantileTransformer'
,'ScaledDataAdjust-PowerTransformer']]#,'diffs']]
# reddit comment data is combined elsewhere
iReleaseData = iReleaseData[['TvShow','RunDate','DailyReleaseCount']]

In [35]:
uberDf2 = pd.concat([instaAccDf,redSubDf,dailyTweetDf,instaHashDf,googDf1,iRedCommentDf])

uberDf3 = MergeDfs(uberDf2, releaseDf,'TvShow')
uberDf3

uberDf3 = MergeDfs(uberDf3, iReleaseData,['TvShow','RunDate'])
uberDf3 = uberDf3[uberDf3['Ignore'] == 1]
uberDf3 = uberDf3.fillna(0)
uberDf3['PostReleaseDay'] = (uberDf3['RunDate'] - uberDf3['Release Date']).astype(str).str.replace(' days','').astype(int)

### Output - individuals & uber dataset

In [36]:
uberDf3 = uberDf3.sort_values(['TvShow','RunDate','SocialMediaSource'])

In [37]:
uberdf4= uberDf3[uberDf3['PostReleaseDay'] <= 90]

In [38]:
uberdf4

,TvShow,RunDate,SocialMediaValue,SocialMediaSource,ScaledDataAdjust-MinMax,ScaledDataAdjust-StandardScaler,ScaledDataAdjust-MaxAbsScaler,ScaledDataAdjust-RobustScaler,ScaledDataAdjust-Normalizer,ScaledDataAdjust-QuantileTransformer,...,Stream,EpisodeCount,SeasonNumber,ReleaseCadence,Release Date,Ignore,ShowStatus,90DayDate,DailyReleaseCount,PostReleaseDay
2150,ABlackLadySketchShow,2022-04-08,17.64,Google,0.176400,0.413910,0.176400,1.335714,1.0,0.824825,...,HBOMax,6.0,3.0,Weekly,2022-04-08,1.0,Renewed,2022-08-06,1.0,0
2147,ABlackLadySketchShow,2022-04-08,175.00,InstagramAccount,0.000783,-0.165021,0.000783,0.155973,1.0,0.582082,...,HBOMax,6.0,3.0,Weekly,2022-04-08,1.0,Renewed,2022-08-06,1.0,0
2149,ABlackLadySketchShow,2022-04-08,12.00,InstagramHashtag,0.000302,-0.168522,0.000302,-0.157895,1.0,0.374875,...,HBOMax,6.0,3.0,Weekly,2022-04-08,1.0,Renewed,2022-08-06,1.0,0
2151,ABlackLadySketchShow,2022-04-08,2.00,RedditComments,0.000057,-0.208537,0.000013,-0.234234,1.0,0.024525,...,HBOMax,6.0,3.0,Weekly,2022-04-08,1.0,Renewed,2022-08-06,1.0,0
2148,ABlackLadySketchShow,2022-04-08,154.00,Tweets,0.000061,-0.186378,0.000061,-0.198792,1.0,0.162703,...,HBOMax,6.0,3.0,Weekly,2022-04-08,1.0,Renewed,2022-08-06,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26259,theEssexSerpent,2022-08-01,50.00,Tweets,0.000020,-0.188050,0.000020,-0.217829,1.0,0.092092,...,Apple,6.0,1.0,Hybrid,2022-05-13,1.0,Completed,2022-09-10,0.0,80
26262,theEssexSerpent,2022-08-02,7.00,InstagramHashtag,0.000176,-0.170753,0.000176,-0.223684,1.0,0.303303,...,Apple,6.0,1.0,Hybrid,2022-05-13,1.0,Completed,2022-09-10,0.0,81
26261,theEssexSerpent,2022-08-02,25.00,Tweets,0.000010,-0.188451,0.000010,-0.222405,1.0,0.057558,...,Apple,6.0,1.0,Hybrid,2022-05-13,1.0,Completed,2022-09-10,0.0,81
26264,theEssexSerpent,2022-08-03,11.00,InstagramHashtag,0.000277,-0.168969,0.000277,-0.171053,1.0,0.363363,...,Apple,6.0,1.0,Hybrid,2022-05-13,1.0,Completed,2022-09-10,0.0,82


filepath2 = r'/Users/cartersocha/Downloads/uberDataset5.csv'

uberdf4.to_csv(filepath2) 

max_tweets_by_show = uberdf4.groupby(['Stream','TvShow','SocialMediaSource']).max('PostReleaseDay')['PostReleaseDay']
max_tweets_df = pd.DataFrame(max_tweets_by_show).reset_index().sort_values('PostReleaseDay', ascending=False)
maxdfff = pd.DataFrame(max_tweets_df.groupby(['TvShow','SocialMediaSource']).max('PostReleaseDay')).reset_index()

maxdfff[maxdfff['PostReleaseDay'] < 70]